#### <font color = "orange">**회사 내부 Confidential 데이터로 판단되는 것은 코드/Output을 삭제하였습니다**

## 01) 전처리

In [1]:
import numpy as np
import pandas as pd

In [4]:
import warnings # 경고 알림 제거
warnings.filterwarnings("ignore", category=Warning) # 경고 알림이 뜨면 모두 무시합니다.

pd.set_option('display.max_columns', 3000)
# pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x) #.0f: 소수점 자리수 지정

### 1) sales & customer data 전처리

#### 1-1) 전처리
- data type 변경
- 비회원 구매 제외 & Jewelry-Watches-Accessories 제품 세일즈만(AS 서비스 거래내역 및 향수 세일즈 제외)
- sales transaction 데이터&고객 데이터 merge

In [ ]:
%%time
# 고객 데이터
df1 = pd.read_excel("DOS_Sales_2021.11.17.xlsx", sheet_name="POWER BI Raw data (Local)", header=10, usecols='B:AD') 
df1.info()

In [ ]:
# Text 타입(Object)를 날짜 형식(Datetime)으로 변환
df1['First Purchase Date'] = pd.to_datetime(df1['First Purchase Date'], yearfirst=True, errors='coerce')
df1['Purchase Date'] = pd.to_datetime(df1['Purchase Date'], yearfirst=True, errors='coerce')
df1['TIME-Year'] = df1['Purchase Date'].dt.year
df1['TIME-Month'] = df1['Purchase Date'].dt.month
df1.head()

In [9]:
df1.shape

(291748, 31)

In [10]:
df1 = df1[~df1['SBU'].isin(['JAS', 'PER', 'WAS', 'AAS'])] # JWA sales only
df1 = df1[~df1['Client Category'].isin(['Employee'])] # w.o employee sales (family sales는 purchase shop에서 ofc로 표기되므로 매장에서 걸러짐)
df1 = df1[(df1['Transaction Type Details']=="Retail-Mchdise sale")|(df1['Transaction Type Details']=="Retail-Return mchdse")] #제품 영수증만(기프트 영수증 및 디파짓 제외)
df1 = df1[~df1['Sales Amount (Purchase Currency)'].isnull()]
df1.shape

(182149, 31)

In [ ]:
StringList = df1['Purchase: Shop'].tolist()
set([p for p in StringList if not(p.startswith(('Bvlgari', 'BVLGARI', 'Bulgari')))]) #우리매장만 골라내기

In [12]:
shop = ["매장 리스트(코드 삭제)"]
len(shop)

22

In [13]:
df1 = df1[df1['Purchase: Shop'].isin(shop)]
df1.shape

(180966, 31)

In [ ]:
df1.reset_index(drop=True, inplace=True)
print(df1.shape)
df1.info()

In [20]:
# 고객 데이터의 세일즈와 확정 세일즈 데이터가 괴리가 클시 확인해볼것
# df1[df1['Purchase Date'].dt.year == 2021].set_index('Purchase Date').groupby(pd.Grouper(freq='M'))['Sales Amount (Purchase Currency)'].sum() / 1.1

In [ ]:
df1.info()

In [ ]:
# df2: 고객데이터 필요한컬럼만 선택
df2 = df1.loc[:, ['TIME-Year', 'TIME-Month', 'Purchase Date', 'Purchase: Shop', 'Main Client Advisor', 'COSMO ID', 'Transaction Type Details', 'Reference Transaction Beanstore2', \
                  'Material Number (SAP Code)', 'Item Quantity', 'Discount %', 'Sales Amount (Purchase Currency)']]\
.rename(columns={'Purchase Date':'TIME-Date', 'Reference Transaction Beanstore2': 'Sales ID BS', 'Material Number (SAP Code)':'MAT_Material(key)', 'Item Quantity':'UNIT SOLD', \
                 'Sales Amount (Purchase Currency)':'NET VALUE'}) #부가세 포함 금액임
print(len(df2))
df2.head()

In [23]:
print(df2['COSMO ID'].nunique())
# df3: 순수 고객데이터베이스
df3 = df1[['COSMO ID', 'Gender', 'Last and First Name (Local)', 'Birthdate Year', 'Segment', 'Nationality', 'First Purchase Date', 'Last Purchase Date General', 'Lifetime Value (User Currency)']] #CRM ID blank생겨서 사용안함
df3.drop_duplicates(subset=['COSMO ID'], keep='last', inplace=True, ignore_index=True)
df3.shape #df2와 고객수 같은지 확인

86854


(86854, 9)

In [ ]:
# 고객데이터 병합
df4 = pd.merge(df2, df3, how='left', on = 'COSMO ID')
df4.info()

#### 1-2) 실제 고객 분석에서 제외할 데이터 추가 편집
1. **리턴하여 실제로 구매하지 않은 제품 삭제**
    - 1. (df7: 티켓별 sum 0)Invoice & 제품 기준 net value sum 0인것 제외 
    - 2. (df8: 고객별 sum < 0) 고객 id & 제품 기준 groupby 한 후 net value sum 0 '미만'인것 제외
    - 3. 고객 id & 제품 & TIME-Date기준 : 같은날 리턴해서 0되는것 제거 
    - 4. 데이터 정렬 후, 리턴 영수증 위에행과 sales 합이 0이면 두 행 제거(리턴 하나전, 리턴 하나 뒤, 리턴 두개 뒤, 리턴 하나전 다시, 리턴 두개 전, 리턴 하나 뒤 다시)
2. 세일즈, 수량 마이너스 고객 제외

In [ ]:
df0 = df4.copy()
print(df0['NET VALUE'].sum())
df0.groupby(['Transaction Type Details'])['Sales ID BS'].nunique() #리턴 영수증 제외전

In [ ]:
# (df7: 티켓별 sum 0)Invoice & 제품 기준 net value sum 0인것 제외 
df7 = df0.groupby(['Sales ID BS', 'MAT_Material(key)'])['NET VALUE'].sum().sort_values().reset_index()
df7 = df7[(df7['NET VALUE'] > -1) & (df7['NET VALUE'] < 1)] #net value sum 0(소수점때문에 범위로) 
df7['key0'] = df7['Sales ID BS'] + df7['MAT_Material(key)']
ticket7 = list(df7['key0'])
df0['key0'] = df0['Sales ID BS'] + df0['MAT_Material(key)']
df0 = df0[~df0['key0'].isin(ticket7)]
print(df0['NET VALUE'].sum()) #영수증 제외 전후 비교

In [ ]:
# (df8: 고객별 sum < 0) 고객 id & 제품 기준 groupby 한 후 net value sum 0 '미만'인것 제외
df8 = df0.groupby(['COSMO ID', 'MAT_Material(key)'])['NET VALUE'].sum().reset_index()
df8 = df8[(df8['NET VALUE'] <= 1)] #-1~1범위로 0 찾음
print(df8[(df8['NET VALUE'] <= 1)]['NET VALUE'].sum()) #제외되는 금액

df8['key'] = df8['COSMO ID'] + df8['MAT_Material(key)']
ticket8 = list(df8['key'])
df0['key'] = df0['COSMO ID'] + df0['MAT_Material(key)']
df0 = df0[~df0['key'].isin(ticket8)]
print(df0['NET VALUE'].sum()) #리턴 금액 제외 후 금액 전후 비교

In [ ]:
#같은날 리턴해서 0되는것 제거
df0 = df0[df0.groupby(['COSMO ID', 'TIME-Date', 'MAT_Material(key)'])["NET VALUE"].transform(sum) != 0]
print(df0['NET VALUE'].sum()) #리턴 편집전과 세일즈 토탈 금액 비교할것 

In [25]:
#리턴과 위에있는 세일즈 0될 경우 지우기 위해 정렬
df0 = df0.sort_values(['COSMO ID', 'MAT_Material(key)', 'TIME-Date', 'Sales ID BS']).reset_index(drop=True)

In [ ]:
%%time
lst = []
pre = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse': 
        if row['NET VALUE'] + df0.loc[index-1, 'NET VALUE'] == 0: #리턴 이전 행과 합이 0
            idx = index
            preidx = index-1
            lst.append(idx)
            pre.append(preidx)
result = lst + pre
df0 = df0[~df0.index.isin(result)]
print(df0['NET VALUE'].sum())

In [ ]:
%%time
df0 = df0.reset_index(drop=True) #인덱스 모두 1차이 나도록 리셋
lst = []
aft = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse':
        if row['NET VALUE'] + df0.loc[index+1, 'NET VALUE'] == 0: #리턴 이후 행과 합이 0
            idx = index
            aftidx = index+1
            lst.append(idx)
            aft.append(aftidx)
result = lst + aft
df0 = df0[~df0.index.isin(result)]
print(df0['NET VALUE'].sum())

In [ ]:
%%time
df0 = df0.reset_index(drop=True) #인덱스 모두 1차이 나도록 리셋
lst = []
aft = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse':
        if row['NET VALUE'] + df0.loc[index+2, 'NET VALUE'] == 0: #리턴 이후 다다음 행과 합이 0 (보통 가격인상전 디파짓걸어놓고 세일즈 클로징할때임)
            idx = index
            aftidx = index+2
            lst.append(idx)
            aft.append(aftidx)
result = lst + aft
df0 = df0[~df0.index.isin(result)]
print(df0['NET VALUE'].sum()) #세일즈는 계속 똑같은지 비교

In [ ]:
%%time
df0 = df0.reset_index(drop=True)
lst = []
pre = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse': 
        if row['NET VALUE'] + df0.loc[index-1, 'NET VALUE'] == 0: #리턴 이전 행과 합이 0 다시한번
            idx = index
            preidx = index-1
            lst.append(idx)
            pre.append(preidx)
result = lst + pre
df0 = df0[~df0.index.isin(result)]
print(df0['NET VALUE'].sum()) #세일즈는 계속 똑같은지 비교

In [ ]:
%%time
df0 = df0.reset_index(drop=True)
lst = []
pre = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse': 
        if row['NET VALUE'] + df0.loc[index-2, 'NET VALUE'] == 0: #리턴 전전 행과 합이 0
            idx = index
            preidx = index-2
            lst.append(idx)
            pre.append(preidx)
result = lst + pre
df0 = df0[~df0.index.isin(result)]
print(df0['NET VALUE'].sum()) #세일즈는 계속 똑같은지 비교

In [ ]:
%%time
df0 = df0.reset_index(drop=True) #인덱스 모두 1차이 나도록 리셋
lst = []
aft = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse':
        if row['NET VALUE'] + df0.loc[index+1, 'NET VALUE'] == 0: #리턴 이후 행과 합이 0
            idx = index
            aftidx = index+1
            lst.append(idx)
            aft.append(aftidx)
result = lst + aft
df0 = df0[~df0.index.isin(result)]
print(df0['NET VALUE'].sum())

In [ ]:
df0.groupby(['Transaction Type Details'])['Sales ID BS'].nunique() #리턴 영수증 거의 다 지움

In [68]:
#영수증 미맵핑 고객아이디 처리 >> groupby 인덱스칼럼에 null있으면 집계결과에 포함 안됨
df0.isnull().sum()

TIME-Year                              0
TIME-Month                             0
TIME-Date                              0
Purchase: Shop                         0
Main Client Advisor                   28
COSMO ID                               0
Transaction Type Details               0
Sales ID BS                            0
MAT_Material(key)                      0
UNIT SOLD                              0
Discount %                        143177
NET VALUE                              0
Gender                              1677
Last and First Name (Local)           55
Birthdate Year                     99092
Segment                             6272
Nationality                        31593
First Purchase Date                  438
Last Purchase Date General           464
Lifetime Value (User Currency)         0
dtype: int64

In [ ]:
df0 = df0.drop(['key0','key'], axis=1) #컬럼 삭제
df0.info()

In [69]:
#영수증 미맵핑 고객아이디 처리 >> groupby 인덱스칼럼에 null있으면 집계결과에 포함 안됨
df0['COSMO ID'] = df0['COSMO ID'].fillna('Not assigned') 
df0.loc[df0[df0['Gender'].isnull()].index, 'Gender'] = 'Not assigned'
df0.loc[df0[df0['Last and First Name (Local)'].isnull()].index, 'Last and First Name (Local)'] = 'Not assigned'
df0.loc[df0[df0['Birthdate Year'].isnull()].index, 'Birthdate Year'] = 'Not assigned'
df0.loc[df0[df0['Segment'].isnull()].index, 'Segment'] = 'Not assigned'
df0.loc[df0[df0['Nationality'].isnull()].index, 'Nationality'] = 'Not assigned'
df0.loc[df0[df0['First Purchase Date'].isnull()].index, 'First Purchase Date'] = 'Not assigned'
df0.loc[df0[df0['Last Purchase Date General'].isnull()].index, 'Last Purchase Date General'] = 'Not assigned'
# df0.loc[df0[df0['Lifetime Value (User Currency)'].isnull()].index, 'Lifetime Value (User Currency)'] = 'Not Assigned'

In [ ]:
df0.info()

In [ ]:
# 마이너스 세일즈, 수량 고객 제외할것
df0.groupby(['COSMO ID', 'Last and First Name (Local)', 'Birthdate Year', 'First Purchase Date']).agg({'UNIT SOLD': 'sum', 'MAT_Material(key)': 'nunique', 'NET VALUE': 'sum'})\
.sort_values('NET VALUE', ascending=True).reset_index()

#### 1-3) 제품 database합치기(**df:sales_transaction_basket_cosmo.xlsx**)

In [78]:
%%time
# 제품 속성 합칠 것
df6 = pd.read_excel('DOS_Sales_2021.11.17.xlsx', sheet_name='ACT')
df6.drop_duplicates(subset=['Sap Code'], keep='last', inplace=True, ignore_index=True)
len(df6)

Wall time: 30.5 s


12869

In [ ]:
df6 = df6.iloc[:, 1:-1] #제품속성만 남김
df6.rename(columns={'Sap Code':'MAT_Material(key)', 'Gender':'MAT_Gender'}, inplace=True)
df0 = pd.merge(df0, df6, how='left', on='MAT_Material(key)') # 제품속성 병합
df0.info()

In [ ]:
df0 = df0[~df0['Short Description'].isnull()] #제품 맵핑 안된것 확인후 지우기
df0.reset_index(drop=True, inplace=True)
df0.info()

In [81]:
%%time
# 제품 카테고리와(SBU) 합칠 것
df9 = pd.read_excel('DOS_Sales_2021.11.17.xlsx', sheet_name='SBU')
df9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   MAT_SBU                 32 non-null     object
 1   MAT_Business Code(key)  32 non-null     object
 2   MAT_Business Code       32 non-null     object
 3   SBU                     32 non-null     object
 4   JWA                     32 non-null     object
 5   MAT_Business Code(num)  32 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 1.6+ KB
Wall time: 13.1 s


In [ ]:
df9 = df9[['MAT_SBU', 'MAT_Business Code(key)', 'SBU', 'JWA']]
df0 = pd.merge(df0, df9, how='left', on='MAT_Business Code(key)') # SBU 병합
df0.info()

In [83]:
%%time
df0.to_excel('sales_transaction_basket_cosmo.xlsx', index=False)

Wall time: 2min 25s
